1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
   - 토큰수 초과로 답변을 생성하지 못할 수 있고
   - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [9]:
!uv add docx2txt langchain-community

Resolved 101 packages in 112ms                                       
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/3.93 KiB            
Prepared 1 package in 35ms                                                   
Installed 1 package in 4ms                                  
 + docx2txt==0.9


In [ ]:
!uv add langchain-text-splitters

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500,
  chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

len(document_list)

183

In [35]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [15]:
!uv add langchain-chroma

Resolved 156 packages in 1.07s                                       
⠙ Preparing packages... (0/54)                                                  
⠙ Preparing packages... (0/54)-------------     0 B/81.19 KiB           
⠙ Preparing packages... (0/54)-------------     0 B/81.19 KiB           
coloredlogs          ------------------------------     0 B/44.94 KiB
⠙ Preparing packages... (0/54)-------------     0 B/81.19 KiB           
coloredlogs          ------------------------------     0 B/44.94 KiB
⠙ Preparing packages... (0/54)-------------     0 B/81.19 KiB           
coloredlogs          ------------------------------     0 B/44.94 KiB
⠙ Preparing packages... (0/54)-------------     0 B/81.19 KiB           
rsa                  ------------------------------     0 B/33.88 KiB
coloredlogs          ------------------------------     0 B/44.94 KiB
⠙ Preparing packages... (0/54)-------------     0 B/81.19 KiB           
rsa                  ------------------------------     0 B/3

In [36]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma-tax", persist_directory="./chroma")
database = Chroma(collection_name="chroma-tax", persist_directory="./chroma", embedding_function=embedding)

In [37]:
query = '연봉 5천만원 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [40]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [41]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

In [42]:
ai_message = llm.invoke(prompt)

In [19]:
ai_message.content

'연봉 5천만원 직장인의 소득세를 계산하기 위해서는 몇 가지 단계가 필요합니다. 여기서는 기본적인 계산 과정을 설명합니다.\n\n1. **과세 표준 계산**:\n   - 연간 총급여: 5천만원\n   - 근로소득공제를 반영하여 과세표준을 계산해야 합니다.\n\n2. **근로소득공제**:\n   - 5천만원의 경우 근로소득공제는 기본적으로 다음과 같은 단계로 계산할 수 있습니다.\n   - 예를 들어, 3,300만원 이하 1,500만원 이상 구간은 740만원 + (총급여 - 1,500) * 22%입니다.\n   - 약간의 변화가 있을 수 있으니 공식적인 세무자료를 참조하시기 바랍니다.\n\n3. **과세표준 세율 적용**:\n   - 과세표준에 해당하는 기본세율을 적용합니다. 대한민국 소득세율은 누진세율을 적용합니다.\n   - 예를 들어, 과세 표준 1200만원 이하에는 6%, 1200만원 초과 4600만원 이하는 15% 등으로 적용합니다.\n\n4. **세액계산 예**:\n   - 예시로 계산하면, 각 구간별로 세율이 적용된 세액을 합산하여 총 소득세를 구합니다.\n\n5. **공제 적용(소득공제 및 세액공제)**:\n   - 기본적으로 적용할 수 있는 소득공제 및 세액공제를 간단히 적용하면 최종 소득세액이 계산됩니다.\n\n연봉이 5천만원인 경우, 다양한 공제 혜택을 적용한 후의 최종 소득세액은 보통 수십만원에서 백만원 단위가 될 수 있습니다. 이 계산은 개략적인 예이며, 정확한 계산을 위해서는 구체적인 공제항목 및 추가적인 소득이 있는지 등의 정보를 바탕으로 세무 전문가에게 문의하거나 전용 소득세 계산기를 활용하는 것이 좋습니다.'

In [20]:
!uv add langchain langchainhub

Resolved 158 packages in 685ms                                       
⠙ Preparing packages... (0/3)                                                   
⠙ Preparing packages... (0/3)--------------     0 B/5.08 KiB            
⠙ Preparing packages... (0/3)---------- 5.08 KiB/5.08 KiB           
⠙ Preparing packages... (0/3)---------- 5.08 KiB/5.08 KiB           
langchainhub         ------------------------------ 5.08 KiB/5.08 KiB
⠙ Preparing packages... (0/3)--------------     0 B/20.16 KiB           
langchainhub         ------------------------------ 5.08 KiB/5.08 KiB
⠙ Preparing packages... (0/3)---------- 14.87 KiB/20.16 KiB         
langchainhub         ------------------------------ 5.08 KiB/5.08 KiB
⠙ Preparing packages... (0/3)---------- 14.87 KiB/20.16 KiB         
langchainhub         ------------------------------ 5.08 KiB/5.08 KiB
⠙ Preparing packages... (0/3)---------- 14.87 KiB/20.16 KiB         
langchainhub         ------------------------------ 5.08 KiB/5.08 KiB
types-re

In [43]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/roy/Projects/2circumflex/inflearn-llm-application-course/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [44]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [45]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=database.as_retriever(),
  chain_type_kwargs={"prompt": prompt}
)

In [46]:
ai_message = qa_chain.invoke({"query": query})

In [33]:
ai_message

{'query': '연봉 5천만원 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원 직장인의 소득세를 계산하려면, 과세표준과 세율을 알아야 하지만 제공된 정보에는 이러한 내용이 포함되어 있지 않습니다. 소득세 계산을 위해서는 소득세법상의 공제사항과 세율표를 참조해야 합니다. 기본적인 과세표준에 대한 내용이 제공되지 않았으므로 정확한 금액을 제시할 수 없습니다.'}